In [ ]:
import boto3
from astropy.io import fits
from io import BytesIO
import sys
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
sys.path.append('/home/bekah/gPhoton2')
from gPhoton.coadd import zero_flag_and_edge

In [ ]:
existing_fuv = 'fuv_1000e.csv'
fuv_objs = pd.read_csv(existing_fuv)
fuv_objs = fuv_objs['eclipse'][0:330]

In [ ]:

def download_and_mask_fits(key, ecl):
    s3 = boto3.client('s3')
    try:
        response = s3.get_object(Bucket='backplanetest', Key=key)
        fits_data = BytesIO(response['Body'].read())

        with fits.open(fits_data) as hdul:
            cnt = hdul[1].data
            flag = hdul[2].data
            edge = hdul[3].data

            masked_cnt = zero_flag_and_edge(cnt, flag, edge)
            
            hdu1 = fits.PrimaryHDU(masked_cnt)
            hdul_new = fits.HDUList([hdu1])

            hdul_new.writeto(f"/media/bekah/BekahA/hotspot/tests/e{ecl}_masked_f.fits", overwrite=True)
        
        print(f"File downloaded, masked, and saved")
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
        else:
            print("Something else went wrong")
            print(str(e))
            
for e in fuv_objs:
    i = str(e).zfill(5)
    key = f"e{i}-fd-WD/e{i}-fd-b00-ffull-image-r.fits"  
    download_and_mask_fits(key, i)

In [ ]:
# make little images 

def linear_stretch(data, stretch_percent=97):
    lower_bound = np.percentile(data, (100 - stretch_percent) / 2)
    upper_bound = np.percentile(data, 100 - (100 - stretch_percent) / 2)
    stretched_data = np.clip((data - lower_bound) / (upper_bound - lower_bound), 0, 1)
    return stretched_data

def list_fits_files(local_dir):
    try:
        return [os.path.join(local_dir, f) for f in os.listdir(local_dir) if f.endswith('.fits')]
    except Exception as e:
        print("Something went wrong while listing files")
        print(str(e))
        return []

def save_fits_as_image(file_path, output_dir, stretch_percent=97):
    try:
        with fits.open(file_path) as hdul:
            for i, hdu in enumerate(hdul):
                if hdu.data is not None:
                    stretched_data = linear_stretch(hdu.data, stretch_percent)
                    image_filename = os.path.join(output_dir, f"{os.path.basename(file_path).replace('.fits', '')}_hdu{i}.png")
                    plt.imsave(image_filename, stretched_data, cmap='gray')
                    print(f"Saved {image_filename}")
    except Exception as e:
        print(f"Something went wrong with file {file_path}")
        print(str(e))

local_dir = '/media/bekah/BekahA/hotspot/tests'
output_dir = '/media/bekah/BekahA/hotspot/tests'

fits_files = list_fits_files(local_dir)

for fits_file in fits_files:
    save_fits_as_image(fits_file, output_dir)
